## Data Description

You are provided with 25,010 poker hands in train.csv and 1,000,000 in test.csv. Each hand consists of five cards with a given suit and rank, drawn from a standard deck of 52. Suits and ranks are represented as ordinal categories:

```
S1 “Suit of card #1”
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}
C1 “Rank of card #1”
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

...

S5 “Suit of card #5”
C5 “Rank of card #5”
```

Each row in the training set has the accompanying class label for the poker hand it comprises. The hands are omitted from the test set and must be predicted by participants. Hands are classified into the following ordinal categories:


```
0: Nothing in hand; not a recognized poker hand 
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush
```
Note that the Straight flush and Royal flush hands are not representative of
the true domain because they have been over-sampled. The straight flush
is 14.43 times more likely to occur in the training set, while the royal flush is 129.82 times more likely.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from functools import reduce
import random

## Utils

#### Test function

In [2]:
def calc_hand_label(hand):    
    def f(hand):
        suits_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,0], {})
        ranks_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,1], {})
        
        if len(ranks_hist.values()) < 5:
            if len(ranks_hist.values()) == 2:
                if max(list(ranks_hist.values())) == 4:
                    return 7
    #                 print('7: Four of a kind; four equal ranks within five cards')
                else:
                    return 6
    #                 print('6 Full house; pair + different rank three of a kind')
            elif len(ranks_hist.values()) == 3:
                if max(list(ranks_hist.values())) == 3:
                    return 3
    #                 print('3: Three of a kind; three equal ranks within five cards')
                else:
                    return 2
    #                 print('2: Two pairs; two pairs of equal ranks within five cards')
            else:
                return 1
    #             print('1: One pair; one pair of equal ranks within five cards')
        else:
            if len(suits_hist.values()) == 1:
                if max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                    if max(list(ranks_hist.keys())) == 13:
                        return 9
    #                     print('9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush')
                    else:
                        return 8
    #                     print('8: Straight flush; straight + flush')
                else:
                    return 5
    #                 print('5: Flush; five cards with the same suit')
            elif max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                return 4    
    #             print('4: Straight; five cards, sequentially ranked with no gaps')
            else:
                return 0
                print('0: Nothing in hand; not a recognized poker hand ')
                
    hand = np.array(hand).reshape(5,2)
    res1 = f(hand)
    
    hand[:,1] = list(map(lambda x: 13 if (x == 1) else x-1, hand[:,1]))
    res2 = f(hand)
    
    if res1 == 9 and res2 == 8:
        return res2
    else:
        return max(res1, res2)

#### Encode cards

In [3]:
def card_encode(s, c):
    cards=["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
    suits="♥♦♣♠"
    return cards[c - 1] + suits[s - 1]

def hand_encode(hand):
    return ', '.join(map(lambda x: card_encode(hand[x*2], hand[x*2+1]),range(5)))

print(hand_encode([4,9,2,1,2,2,4,7,2,8]))
print(hand_encode([2,9,2,4,3,6,1,9,4,9]))

9♠, A♦, 2♦, 7♠, 8♦
9♦, 4♦, 6♣, 9♥, 9♠


## Train data

In [4]:
def shuffle_df(data):
    data = data.copy()
    for index, row in data.iterrows():
        rank = -1
    
        row['S1'] = random.randint(1,4)
        row['S2'] = random.randint(1,4)
        row['S3'] = random.randint(1,4)
        row['S4'] = random.randint(1,4)
        row['S5'] = random.randint(1,4)
        row['C1'] = random.randint(1,13)
        row['C2'] = random.randint(1,13)
        row['C3'] = random.randint(1,13)
        row['C4'] = random.randint(1,13)
        row['C5'] = random.randint(1,13)
        row['hand'] = calc_hand_label(row[['S1', 'C1','S2', 'C2','S3', 'C3','S4', 'C4','S5', 'C5']])
    return data

In [5]:
suits = ['S1','S2','S3','S4','S5']
cards = ['C1','C2','C3','C4','C5']
distances = ['D1', 'D2', 'D3', 'D4', 'D5']

In [6]:
train_dataset = pd.read_csv('train.csv', delimiter=',')

In [7]:
train_dataset = train_dataset\
.append(shuffle_df(train_dataset),ignore_index=True)\
.append(shuffle_df(train_dataset),ignore_index=True)\
.append(shuffle_df(train_dataset),ignore_index=True)\
.append(shuffle_df(train_dataset),ignore_index=True)\
.append(shuffle_df(train_dataset),ignore_index=True)\
.append(shuffle_df(train_dataset),ignore_index=True)

In [8]:
train_dataset.shape

(175070, 11)

In [9]:
data = train_dataset.drop('hand', axis=1)
train_labels = train_dataset['hand']

In [10]:
labels_vect = tf.keras.utils.to_categorical(train_labels)

#### Adding distance features

In [11]:
def get_distances(hand):
    distances_list = []
    for i, card in enumerate(hand):
        if (i < len(hand) - 1):
            distances_list.append(hand[i+1] - card)
        else:
            distances_list.append(13 - card + hand[0])
    return pd.Series(distances_list)

def with_distances(data):
    sorted = data[cards]
    sorted.values.sort()

    distances_df = sorted.apply(get_distances, axis=1)
    distances_df.columns = distances
    
    data_with_distances = pd.concat([data, distances_df], axis=1)
    return data_with_distances

In [12]:
data_with_distances = with_distances(data)
data_with_distances.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,D1,D2,D3,D4,D5
0,4,9,2,1,2,2,4,7,2,8,1,5,1,1,5
1,1,4,3,6,1,12,3,11,2,7,2,1,4,1,5
2,1,11,4,1,3,7,4,11,2,1,0,6,4,0,3
3,2,9,2,4,3,6,1,9,4,9,2,3,0,0,8
4,1,8,2,4,2,11,2,2,2,1,1,2,4,3,3


#### One hot encoding

In [13]:
def one_hot_encoded(data, columns, encoder, mode='transform'):
    encoded = None
    if (mode == 'fit'):
        encoded = encoder.fit_transform(data[columns])
    elif (mode == 'transform'):
        encoded = encoder.transform(data[columns])
    return pd.concat([data.drop(columns, axis=1), pd.DataFrame(encoded.toarray())], axis=1)

In [14]:
encoder = OneHotEncoder()
data_encoded = one_hot_encoded(data_with_distances, suits, encoder, 'fit')
print(data_encoded.shape)

data_encoded.head()

(175070, 30)


,C1,C2,C3,C4,C5,D1,D2,D3,D4,D5,...,10,11,12,13,14,15,16,17,18,19
0,9,1,2,7,8,1,5,1,1,5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,4,6,12,11,7,2,1,4,1,5,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,11,1,7,11,1,0,6,4,0,3,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,9,4,6,9,9,2,3,0,0,8,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,8,4,11,2,1,1,2,4,3,3,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Normalization

In [15]:
def normalized(data, columns, mean, std):
    data_normalized = (data[columns] - mean) / std
    to_return = pd.concat([data_normalized, data.drop(columns, axis=1)], axis=1)
    return to_return

In [16]:
mean_c = data_encoded[cards].mean().mean()
std_c = pd.Series(data_encoded[cards].values.flatten()).std()
mean_d = data_encoded[distances].mean().mean()
std_d = pd.Series(data_encoded[distances].values.flatten()).std()

data_normalized = normalized(data_encoded, cards, mean_c, std_c)
data_normalized = normalized(data_normalized, distances, mean_d, std_d)
data_normalized.head()

,D1,D2,D3,D4,D5,C1,C2,C3,C4,C5,...,10,11,12,13,14,15,16,17,18,19
0,-0.744891,1.117337,-0.744891,-0.744891,1.117337,0.536178,-1.598710,-1.331849,0.002456,0.269317,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,-0.279334,-0.744891,0.651780,-0.744891,1.117337,-0.798127,-0.264405,1.336761,1.069900,0.002456,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,-1.210448,1.582894,0.651780,-1.210448,0.186223,1.069900,-1.598710,0.002456,1.069900,-1.598710,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,-0.279334,0.186223,-1.210448,-1.210448,2.514008,0.536178,-0.798127,-0.264405,0.536178,0.536178,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.744891,-0.279334,0.651780,0.186223,0.186223,0.269317,-0.798127,1.069900,-1.331849,-1.598710,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
data_normalized.shape

(175070, 30)

## Neural Network

In [22]:
from time import time
from keras.callbacks import TensorBoard

model = keras.Sequential()
#input
model.add(keras.layers.Dense(120, activation='relu', input_shape=(30,)))
model.add(keras.layers.Dropout(0.2))
#hidden
model.add(keras.layers.Dense(240, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(120, activation='relu'))
model.add(keras.layers.Dropout(0.1))
model.add(keras.layers.Dense(120, activation='relu'))
model.add(keras.layers.Dense(60, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(keras.layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(np.array(data_normalized), np.array(labels_vect), epochs=500, batch_size=512)

Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/500
175070/175070 [==============================]175070/175070 [==============================] - 5s 28us/step - loss: 0.9610 - acc: 0.5970

Epoch 2/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 0.2888 - acc: 0.8997

Epoch 3/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 25us/step - loss: 0.0616 - acc: 0.9832

Epoch 4/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 0.0311 - acc: 0.9926

Epoch 5/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 0.0223 - acc: 0.9953

Epoch 6/500
175070/175070 [==============================]175070/175070 [==============================]

175070/175070 [==============================]175070/175070 [==============================] - 5s 29us/step - loss: 0.0012 - acc: 0.9997

Epoch 51/500
175070/175070 [==============================]175070/175070 [==============================] - 6s 36us/step - loss: 0.0012 - acc: 0.9997

Epoch 52/500
175070/175070 [==============================]175070/175070 [==============================] - 5s 26us/step - loss: 9.2013e-04 - acc: 0.9998

Epoch 53/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 7.7278e-04 - acc: 0.9998

Epoch 54/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 6.1513e-04 - acc: 0.9999

Epoch 55/500
175070/175070 [==============================]175070/175070 [==============================] - 5s 26us/step - loss: 8.7825e-04 - acc: 0.9998

Epoch 56/500
175070/175070 [==============================]175070/175070 [=================

175070/175070 [==============================]175070/175070 [==============================] - 4s 25us/step - loss: 6.1834e-04 - acc: 0.9999/175070 [..............................] - ETA: 3s - loss: 7.8480e-04 - acc: 0.999  8192/175070 [>

Epoch 104/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 23us/step - loss: 6.9052e-04 - acc: 0.9998

Epoch 105/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 25us/step - loss: 4.2771e-04 - acc: 0.9999

Epoch 106/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 22us/step - loss: 4.6413e-04 - acc: 0.9999

Epoch 107/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 23us/step - loss: 6.7135e-04 - acc: 0.9999

Epoch 108/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 23us/step - loss: 5.7014


Epoch 155/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 25us/step - loss: 4.1186e-04 - acc: 0.9999

Epoch 156/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 23us/step - loss: 4.9897e-04 - acc: 0.9999

Epoch 157/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 5.9658e-04 - acc: 0.9999

Epoch 158/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 4.4365e-04 - acc: 0.9999

Epoch 159/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 2.6586e-04 - acc: 1.0000

Epoch 160/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 3.9938e-04 - acc: 1.0000/175070 [>...

Epoch 161/500
175070/175070 [=====================

...

175070/175070 [==============================]175070/175070 [==============================] - 5s 26us/step - loss: 3.3306e-04 - acc: 1.0000

Epoch 260/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 4.3490e-04 - acc: 0.9999

Epoch 261/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 8.7147e-04 - acc: 0.9999

Epoch 262/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 3.9324e-04 - acc: 0.9999

Epoch 263/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 3.7956e-04 - acc: 0.9999

Epoch 264/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 4.4202e-04 - acc: 0.9999

Epoch 265/500
175070/175070 [==============================]175070/175070 [===

175070/175070 [==============================]175070/175070 [==============================] - 5s 30us/step - loss: 3.0173e-04 - acc: 1.0000

Epoch 312/500
175070/175070 [==============================]175070/175070 [==============================] - 5s 29us/step - loss: 4.6062e-04 - acc: 0.9999

Epoch 313/500
175070/175070 [==============================]175070/175070 [==============================] - 5s 26us/step - loss: 3.1972e-04 - acc: 1.0000

Epoch 314/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 3.4895e-04 - acc: 0.9999

Epoch 315/500
175070/175070 [==============================]175070/175070 [==============================] - 6s 36us/step - loss: 4.5015e-04 - acc: 0.9999

Epoch 316/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 3.6333e-04 - acc: 1.0000

Epoch 317/500
175070/175070 [==============================]175070/175070 [===

175070/175070 [==============================]175070/175070 [==============================] - 6s 36us/step - loss: 4.4925e-04 - acc: 0.9999: 0s - loss: 3.7883e-04 - acc:

Epoch 364/500
175070/175070 [==============================]175070/175070 [==============================] - 7s 38us/step - loss: 5.8759e-04 - acc: 0.9999

Epoch 365/500
175070/175070 [==============================]175070/175070 [==============================] - 6s 33us/step - loss: 4.5102e-04 - acc: 1.0000

Epoch 366/500
175070/175070 [==============================]175070/175070 [==============================] - 6s 34us/step - loss: 8.5220e-04 - acc: 0.9999

Epoch 367/500
175070/175070 [==============================]175070/175070 [==============================] - 6s 34us/step - loss: 4.1891e-04 - acc: 1.0000

Epoch 368/500
175070/175070 [==============================]175070/175070 [==============================] - 6s 32us/step - loss: 2.7084e-04 - acc: 1.0000

Epoch 369/500
175070/175070 [===================

175070/175070 [==============================]175070/175070 [==============================] - 6s 37us/step - loss: 4.6702e-04 - acc: 0.9999

Epoch 416/500
175070/175070 [==============================]175070/175070 [==============================] - 7s 40us/step - loss: 2.7507e-04 - acc: 1.0000

Epoch 417/500
175070/175070 [==============================]175070/175070 [==============================] - 5s 31us/step - loss: 2.5254e-04 - acc: 1.0000

Epoch 418/500
175070/175070 [==============================]175070/175070 [==============================] - 6s 34us/step - loss: 4.5862e-04 - acc: 0.9999

Epoch 419/500
175070/175070 [==============================]175070/175070 [==============================] - 6s 36us/step - loss: 4.0400e-04 - acc: 1.0000

Epoch 420/500
175070/175070 [==============================]175070/175070 [==============================] - 7s 39us/step - loss: 3.7558e-04 - acc: 1.0000

Epoch 421/500
175070/175070 [==============================]175070/175070 [===

175070/175070 [==============================]175070/175070 [==============================] - 4s 25us/step - loss: 4.8081e-04 - acc: 0.9999

Epoch 468/500
175070/175070 [==============================]175070/175070 [==============================] - 5s 28us/step - loss: 5.9187e-04 - acc: 0.9999

Epoch 469/500
175070/175070 [==============================]175070/175070 [==============================] - 5s 30us/step - loss: 3.6008e-04 - acc: 0.9999

Epoch 470/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 3.9201e-04 - acc: 1.0000

Epoch 471/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 6.3183e-04 - acc: 0.9999

Epoch 472/500
175070/175070 [==============================]175070/175070 [==============================] - 4s 24us/step - loss: 2.4441e-04 - acc: 1.0000

Epoch 473/500
175070/175070 [==============================]175070/175070 [===

In [26]:
model.save('./model-20')

## Test data

In [18]:
test_dataset = pd.read_csv('test.csv', delimiter=',')
test_data = test_dataset.drop('id', axis=1)

In [19]:
test_labels = test_data.apply(calc_hand_label, axis=1)

In [20]:
test_data_with_distances = with_distances(test_data)
test_data_encoded = one_hot_encoded(test_data_with_distances, suits, encoder)
test_data_normalized = normalized(test_data_encoded, cards, mean_c, std_c)
test_data_normalized = normalized(test_data_normalized, distances, mean_d, std_d)

In [21]:
test_data_normalized.shape

(1000000, 30)

## Predictions

In [23]:
test_data_sample_predicted_10 = model.predict(np.array(test_data_normalized))
test_data_sample_predicted = np.array(list(map(np.argmax, test_data_sample_predicted_10)))

In [24]:
predictions_data = pd.concat([
    test_dataset,
    pd.DataFrame({'predictions': test_data_sample_predicted}),
    pd.DataFrame({'labels': test_labels}),
], axis=1)

In [25]:
wrong_answers = predictions_data[predictions_data['predictions'] != predictions_data['labels']]
wrong_answers.shape

(20, 13)

In [27]:
wrong_answers['labels'].value_counts()

0    20
Name: labels, dtype: int64

In [28]:
wrong_answers

,id,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,predictions,labels
20465,20466,3,9,3,11,2,1,1,13,2,10,4,0
94531,94532,1,9,4,11,2,10,1,1,1,13,4,0
117681,117682,3,11,1,10,1,1,1,13,1,9,4,0
243854,243855,3,12,2,13,1,1,4,10,2,9,4,0
266363,266364,3,10,2,13,2,1,2,11,2,9,4,0
311641,311642,1,13,2,12,2,2,1,11,2,10,4,0
344104,344105,3,10,4,11,2,1,3,13,1,9,4,0
392442,392443,4,10,4,13,2,1,1,9,1,11,4,0
423814,423815,3,10,3,13,2,1,4,11,3,9,4,0
435313,435314,3,13,3,10,2,1,3,9,2,12,4,0


In [29]:
wrong_answers.drop(['id', 'predictions', 'labels'], axis=1).apply(lambda h: hand_encode(np.array(h)), axis=1)

20465     9♣, J♣, A♦, K♥, 10♦
94531     9♥, J♠, 10♦, A♥, K♥
117681    J♣, 10♥, A♥, K♥, 9♥
243854    Q♣, K♦, A♥, 10♠, 9♦
266363    10♣, K♦, A♦, J♦, 9♦
311641    K♥, Q♦, 2♦, J♥, 10♦
344104    10♣, J♠, A♦, K♣, 9♥
392442    10♠, K♠, A♦, 9♥, J♥
423814    10♣, K♣, A♦, J♠, 9♣
435313    K♣, 10♣, A♦, 9♣, Q♦
458662    9♣, J♣, A♦, K♥, 10♦
592868    A♥, 10♠, J♠, 9♦, K♥
661709    10♣, 9♠, A♦, Q♥, K♥
705362    K♦, J♣, A♥, 10♠, 9♥
739262    K♦, 9♣, A♥, J♥, 10♥
750826    K♣, 10♠, 2♦, Q♥, J♣
753542    10♣, K♣, A♦, 9♥, J♣
763420    K♣, 10♦, 2♦, Q♠, J♦
877353    10♦, K♠, 9♦, A♥, Q♥
900270    K♦, Q♠, A♦, 9♥, 10♣
dtype: object